In [436]:
import numpy as np
import pandas as pd
from datascience import *



In [437]:
LDLR_file_path = '/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/LDLR_table_refseq.txt'
SREBF2_file_path = '/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/SREBF2_table_refseq.txt'
HNF4A_file_path = '/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/HNF4A_table_refseq.txt'
MYLIP_file_path = '/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/MYLIP_table_refseq.txt'
ACAT2_file_path = '/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/ACAT2_table_refseq.txt'
LSS_file_path = '/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/LSS_table_refseq.txt'
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [459]:
LDLR_sequence_path = "/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/sequence_files/LDLR_sequence_path.txt"
SREBF2_sequence_path = "/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/sequence_files/SREBF2_sequence_path.txt"
HNF4A_sequence_path = "/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/sequence_files/HNF4A_sequence_path.txt"
MYLIP_sequence_path = "/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/sequence_files/MYLIP_sequence_path.txt"
ACAT2_sequence_path = "/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/sequence_files/ACAT2_sequence_path.txt"
LSS_sequence_path = "/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/sequence_files/LSS_sequence_path2.txt"



In [439]:
def sequence_table(gene, sequence_file_path, gene_file_path):
    """returns table with the sequence and reading frame of each exon. Inputs are exon sequence file and RefGene file from UCSC table browser."""
    seq_file = open(sequence_file_path).read()
    genes = seq_file.split(">")
    del genes[0]
    gene = []
    sequence = []
    for i in genes:
        gene = np.append(gene, i.split("=none\n")[0])
        sequence = np.append(sequence, i.split("=none\n")[1].replace("\n", ""))
    gene_file = pd.read_csv(gene_file_path, sep = "\t").loc[0]
    reading_frames = gene_file.loc["exonFrames"].strip(",").split(",")
    exon_starts = gene_file.loc["exonStarts"].strip(",").split(",")
    exon_ends = gene_file.loc["exonEnds"].strip(",").split(",")
    if reading_frames[0]=='-1':
        del reading_frames[0]
        del exon_starts[0]
        del exon_ends[0]
    seq_table = pd.DataFrame({"GeneID": gene, "chromosome": gene_file.loc["chrom"].strip("chr"), "strand": gene_file.loc["strand"], "sequence": sequence, "reading_frame":reading_frames, "exon_start": exon_starts, "exon_end": exon_ends})
    return seq_table

a = open(LSS_sequence_path).read()
b = a.split(">")
del b[0]
c = []
d = []
for i in b:
    c = np.append(c, i.split("=none\n")[0])
    d = np.append(d, i.split("=none\n")[1].replace("\n", ""))
gene_filed = pd.read_csv(LSS_file_path, sep = "\t").loc[0]
reading_framesd = gene_filed.loc["exonFrames"].strip(",").split(",")
exon_startsd = gene_filed.loc["exonStarts"].strip(",").split(",")
exon_endsd = gene_filed.loc["exonEnds"].strip(",").split(",")
if reading_framesd[0]=='-1':
    del reading_framesd[0]
    del exon_startsd[0]
    del exon_endsd[0]

#reading_framesd[0]=='-1'
print(len(exon_startsd))
print(len(exon_endsd))
print(len(reading_framesd))
print(len(c))
print(len(d))
b

In [440]:

rv_target_codons = ["CGA", "TGG"]
fw_target_codons = ["CGA", "CAA", "CAG"]


target G's in rv and C's in fw

In [441]:
def stop_gain_targets(gene, sequence_file_path, gene_file_path):
    seq_table = sequence_table(gene, sequence_file_path, gene_file_path).drop(0)
    rv_target_codons = ["TGG", "CGA"]
    fw_target_codons = ["CGA", "CAA", "CAG"]
    guide1s = []
    guide2s = []
    guide3s = []
    guide4s = []
    guide5s = []
    target_strand = []
    target_codons = []
    target_exon = []
    reading_frames = []
    for i in np.arange(1, len(seq_table)):
        row = seq_table.loc[i]
        exon = i+1
        reading_frame = int(row.loc["reading_frame"])
        sequence = row.loc["sequence"]
        if reading_frame == -1:
            in_frame_sequence = sequence[2:len(sequence)]
            adjust = 3
        else:
            in_frame_sequence = sequence[reading_frame:len(sequence)+1]
            adjust = 0
        codons = []
        for i in np.arange(0,len(in_frame_sequence), 3):
            codon = in_frame_sequence[i+reading_frame:(i+reading_frame+3)]
            codons = np.append(codons, codon)
            if codon in fw_target_codons:
                target_base = i + int(row.loc["exon_start"]) + adjust + reading_frame
                guide_1_start = target_base+reading_frame-2
                guide_1_end = guide_1_start+19
                guide_2_start = guide_1_start - 1
                guide_2_end = guide_2_start + 19
                guide_3_start = guide_1_start - 2
                guide_3_end = guide_3_start + 19
                guide_4_start = guide_1_start - 3
                guide_4_end = guide_4_start + 19
                guide_5_start = guide_1_start - 4
                guide_5_end = guide_5_start + 19
                target_strand = np.append(target_strand, row.loc["strand"])
                guide1s = np.append(guide1s, "chr"+str(row.loc["chromosome"])+":"+str(guide_1_start)+"-"+str(guide_1_end))
                guide2s = np.append(guide2s, "chr"+str(row.loc["chromosome"])+":"+str(guide_2_start)+"-"+str(guide_2_end))
                guide3s = np.append(guide3s, "chr"+str(row.loc["chromosome"])+":"+str(guide_3_start)+"-"+str(guide_3_end))
                guide4s = np.append(guide4s, "chr"+str(row.loc["chromosome"])+":"+str(guide_4_start)+"-"+str(guide_4_end))
                guide5s = np.append(guide5s, "chr"+str(row.loc["chromosome"])+":"+str(guide_5_start)+"-"+str(guide_5_end))
                target_codons = np.append(target_codons, codon)
                target_exon = np.append(target_exon, exon)
                reading_frames = np.append(reading_frames, reading_frame)
            if codon in rv_target_codons:
                target_base = i + int(row.loc["exon_start"]) + adjust + reading_frame
                guide_1_end = target_base + reading_frame + 5
                guide_1_start = guide_1_end-19
                guide_2_start = guide_1_start + 1
                guide_2_end = guide_1_end + 1
                guide_3_start = guide_1_start + 2
                guide_3_end = guide_1_end + 2
                guide_4_start = guide_1_start + 3
                guide_4_end = guide_1_end + 3
                guide_5_start = guide_1_start + 4
                guide_5_end = guide_1_end + 4
                target_codons = np.append(target_codons, codon)
                target_exon = np.append(target_exon, exon)
                reading_frames = np.append(reading_frames, reading_frame)
                if row.loc["strand"] == "+":
                    target_strand = np.append(target_strand, "-")
                else:
                    target_strand = np.append(target_strand, "+")
                guide1s = np.append(guide1s, "chr"+str(row.loc["chromosome"])+":"+str(guide_1_start)+"-"+str(guide_1_end))
                guide2s = np.append(guide2s, "chr"+str(row.loc["chromosome"])+":"+str(guide_2_start)+"-"+str(guide_2_end))
                guide3s = np.append(guide3s, "chr"+str(row.loc["chromosome"])+":"+str(guide_3_start)+"-"+str(guide_3_end))
                guide4s = np.append(guide4s, "chr"+str(row.loc["chromosome"])+":"+str(guide_4_start)+"-"+str(guide_4_end))
                guide5s = np.append(guide5s, "chr"+str(row.loc["chromosome"])+":"+str(guide_5_start)+"-"+str(guide_5_end))
            else:
                continue

    target_table = pd.DataFrame({"gene": gene, "site": "stop/gain", "exon":target_exon, "strand": target_strand, "BE": "CBE", "target_codon": target_codons, "reading_frame": reading_frames, "guide1":guide1s , "guide2":guide2s, "guide3":guide3s, "guide4":guide4s, "guide5":guide5s})
    return target_table[target_table["exon"]!=1]
        

In [442]:
LDLR = stop_gain_targets("LDLR", LDLR_sequence_path, LDLR_file_path)
#w[w["reading_frame"]!=0]
#or exon !=0?

In [457]:
SREBF2 = stop_gain_targets("SREBF2", SREBF2_sequence_path, SREBF2_file_path)


In [444]:
sequence_table("HNF4A", HNF4A_sequence_path, HNF4A_file_path)

,GeneID,chromosome,strand,sequence,reading_frame,exon_start,exon_end
0,hg38_refGene_NM_001287184_0 range=chr20:443906...,20,+,ATGTCGGACTGGGGCCAGGGCTTCCCCCAGGACCCACCAG,0,44390457,44390679
1,hg38_refGene_NM_001287184_1 range=chr20:444060...,20,+,ACACGTCCCCATCAGAAGGCACCAACCTCAACGCGCCCAACAGCCT...,1,44406057,44406232
2,hg38_refGene_NM_001287184_2 range=chr20:444073...,20,+,ATTTAGCCGGCAGTGCGTGGTGGACAAAGACAAGAGGAACCAGTGC...,2,44407380,44407475
3,hg38_refGene_NM_001287184_3 range=chr20:444136...,20,+,CCGTCCAGAATGAGCGGGACCGGATCAGCACTCGAAGGTCAAGCTA...,1,44413693,44413800
4,hg38_refGene_NM_001287184_4 range=chr20:444145...,20,+,ATCACCTCCCCCGTCTCCGGGATCAACGGCGACATTCGGGCGAAGA...,0,44414506,44414662
5,hg38_refGene_NM_001287184_5 range=chr20:444184...,20,+,GTGGCCCTGCTCAGAGCCCATGCTGGCGAGCACCTGCTGCTCGGAG...,0,44418424,44418512
6,hg38_refGene_NM_001287184_6 range=chr20:444197...,20,+,GCAATGACTACATTGTCCCTCGGCACTGCCCGGAGCTGGCGGAGAT...,1,44419720,44419876
7,hg38_refGene_NM_001287184_7 range=chr20:444240...,20,+,ATGCCAAGGGGCTGAGCGATCCAGGGAAGATCAAGCGGCTGCGTTC...,1,44424017,44424643


In [455]:
HNF4A = stop_gain_targets("HNF4A", HNF4A_sequence_path, HNF4A_file_path)


In [453]:
MYLIP = stop_gain_targets("MYLIP", MYLIP_sequence_path, MYLIP_file_path)


In [454]:
ACAT2 = stop_gain_targets("ACAT2", ACAT2_sequence_path, ACAT2_file_path)


In [461]:
LSS = stop_gain_targets("LSS", LSS_sequence_path, LSS_file_path)
sequence_table("LSS", LSS_sequence_path, LSS_file_path)

,GeneID,chromosome,strand,sequence,reading_frame,exon_start,exon_end
0,hg38_refGene_NM_001001438_0 range=chr21:462287...,21,-,ATGACGGAGGGCAC,0,46191065,46191235
1,hg38_refGene_NM_001001438_1 range=chr21:462284...,21,-,GTGTCTGCGGCGCCGAGGGGGCCCCTACAAGACCGAGCCCGCCACC...,2,46191880,46191959
2,hg38_refGene_NM_001001438_2 range=chr21:462275...,21,-,AAGAATTACTTTAAGGACTTGCCCAAAGCCCACACCGCCTTTGAGG...,2,46194490,46194661
3,hg38_refGene_NM_001001438_3 range=chr21:462226...,21,-,GCCTCCTGATCACTTGCCACGTGGCACGCATCCCTCTGCCAGCCGG...,2,46195675,46195756
4,hg38_refGene_NM_001001438_4 range=chr21:462218...,21,-,GCACATTGAGGATAAGTCCACCGTGTTTGGGACTGCGCTCAACTAT...,2,46196201,46196267
5,hg38_refGene_NM_001001438_5 range=chr21:462194...,21,-,GTGGTGCTGTGGCCATCCCCTCCTGGGGGAAGTTCTGGCTGGCTGT...,1,46205835,46205941
6,hg38_refGene_NM_001001438_6 range=chr21:462163...,21,-,GCTGTTTCCTGACTGGGCACCGGCACACCCCTCCACACTCTGGTGC...,0,46206671,46206768
7,hg38_refGene_NM_001001438_7 range=chr21:462156...,21,-,GAGCTCTATGTGGAGGACTTCGCCAGCATTGACTGGCTGGCGCAGA...,0,46207427,46207577
8,hg38_refGene_NM_001001438_8 range=chr21:462151...,21,-,CGCTCCTCAACCTGTATGAGCACCACCACAGTGCCCACCTGCGGCA...,0,46208250,46208301
9,hg38_refGene_NM_001001438_9 range=chr21:462137...,21,-,ATCTCGAAAACCATCAACATGCTTGTGCGCTGGTATGTGGACGGGC...,0,46209553,46209625


In [462]:
LSS

,gene,site,exon,strand,BE,target_codon,reading_frame,guide1,guide2,guide3,guide4,guide5
0,LSS,stop/gain,2.0,-,CBE,CGA,2.0,chr21:46191891-46191910,chr21:46191890-46191909,chr21:46191889-46191908,chr21:46191888-46191907,chr21:46191887-46191906
1,LSS,stop/gain,2.0,+,CBE,CGA,2.0,chr21:46191879-46191898,chr21:46191880-46191899,chr21:46191881-46191900,chr21:46191882-46191901,chr21:46191883-46191902
2,LSS,stop/gain,2.0,+,CBE,TGG,2.0,chr21:46191924-46191943,chr21:46191925-46191944,chr21:46191926-46191945,chr21:46191927-46191946,chr21:46191928-46191947
3,LSS,stop/gain,2.0,-,CBE,CGA,2.0,chr21:46191939-46191958,chr21:46191938-46191957,chr21:46191937-46191956,chr21:46191936-46191955,chr21:46191935-46191954
4,LSS,stop/gain,2.0,+,CBE,CGA,2.0,chr21:46191927-46191946,chr21:46191928-46191947,chr21:46191929-46191948,chr21:46191930-46191949,chr21:46191931-46191950
5,LSS,stop/gain,2.0,-,CBE,CAG,2.0,chr21:46191963-46191982,chr21:46191962-46191981,chr21:46191961-46191980,chr21:46191960-46191979,chr21:46191959-46191978
6,LSS,stop/gain,2.0,+,CBE,TGG,2.0,chr21:46191957-46191976,chr21:46191958-46191977,chr21:46191959-46191978,chr21:46191960-46191979,chr21:46191961-46191980
7,LSS,stop/gain,2.0,-,CBE,CAG,2.0,chr21:46191981-46192000,chr21:46191980-46191999,chr21:46191979-46191998,chr21:46191978-46191997,chr21:46191977-46191996
8,LSS,stop/gain,2.0,-,CBE,CAG,2.0,chr21:46192005-46192024,chr21:46192004-46192023,chr21:46192003-46192022,chr21:46192002-46192021,chr21:46192001-46192020
9,LSS,stop/gain,3.0,+,CBE,TGG,2.0,chr21:46194549-46194568,chr21:46194550-46194569,chr21:46194551-46194570,chr21:46194552-46194571,chr21:46194553-46194572


In [458]:
#LDLR.to_csv('/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/LDLR_CBE_controls.csv')
#SREBF2.to_csv('/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/SEBF2_CBE_controls.csv')
#HNF4A.to_csv('/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/HNF4A_CBE_controls.csv')
#MYLIP.to_csv('/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/MYLIP_CBE_controls.csv')
#ACAT2.to_csv('/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/ACAT2_CBE_controls.csv')
LSS.to_csv('/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/LSS_CBE_controls.csv')

